In [ ]:
#check path # for dev opnly
!pwd
!nvidia-smi

<h1><b><center>Process 1/2: Text To Art

In [ ]:
#@title <center> Setup block
#@markdown <center>Run Once


print(" starting 2. Downloading and Installation of libraries")

print("Downloading CLIP...")
!git clone https://github.com/openai/CLIP                 &> /dev/null
 
print("Installing Python Libraries for AI...")
!git clone https://github.com/CompVis/taming-transformers &> /dev/null
!pip install ftfy regex tqdm omegaconf pytorch-lightning  &> /dev/null
!pip install kornia                                       &> /dev/null
!pip install einops                                       &> /dev/null
!pip install wget                                         &> /dev/null
 
print("Installing libraries for handling metadata...")
!pip install stegano                                      &> /dev/null
!apt install exempi                                       &> /dev/null
!pip install python-xmp-toolkit                           &> /dev/null
!pip install imgtag                                       &> /dev/null
!pip install pillow==7.1.2                                &> /dev/null
 
print("Installing Python libraries for creating videos ...")
!pip install imageio-ffmpeg                               &> /dev/null
!mkdir steps
print("Step 2. done")


print("3. models to download")
!curl -L -o vqgan_imagenet_f16_16384.yaml -C - 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' #ImageNet 16384
!curl -L -o vqgan_imagenet_f16_16384.ckpt -C - 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fckpts%2Flast.ckpt&dl=1' #ImageNet 16384
print("Step 3. done")



print("4. Loading Libraries and definitions")
 
import argparse
import math
from pathlib import Path
import sys
 
sys.path.append('./taming-transformers')
from IPython import display
from base64 import b64encode
from omegaconf import OmegaConf
from PIL import Image
from taming.models import cond_transformer, vqgan
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from tqdm.notebook import tqdm
 
from CLIP import clip
import kornia.augmentation as K
import numpy as np
import imageio
from PIL import ImageFile, Image
from imgtag import ImgTag    # metadatos 
from libxmp import *         # metadatos
import libxmp                # metadatos
from stegano import lsb
import json
ImageFile.LOAD_TRUNCATED_IMAGES = True
 
def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))
 
 
def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()
 
 
def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]
 
 
def resample(input, size, align_corners=True):
    n, c, h, w = input.shape
    dh, dw = size
 
    input = input.view([n * c, 1, h, w])
 
    if dh < h:
        kernel_h = lanczos(ramp(dh / h, 2), 2).to(input.device, input.dtype)
        pad_h = (kernel_h.shape[0] - 1) // 2
        input = F.pad(input, (0, 0, pad_h, pad_h), 'reflect')
        input = F.conv2d(input, kernel_h[None, None, :, None])
 
    if dw < w:
        kernel_w = lanczos(ramp(dw / w, 2), 2).to(input.device, input.dtype)
        pad_w = (kernel_w.shape[0] - 1) // 2
        input = F.pad(input, (pad_w, pad_w, 0, 0), 'reflect')
        input = F.conv2d(input, kernel_w[None, None, None, :])
 
    input = input.view([n, c, h, w])
    return F.interpolate(input, size, mode='bicubic', align_corners=align_corners)
 
 
class ReplaceGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_forward, x_backward):
        ctx.shape = x_backward.shape
        return x_forward
 
    @staticmethod
    def backward(ctx, grad_in):
        return None, grad_in.sum_to_size(ctx.shape)
 
 
replace_grad = ReplaceGrad.apply
 
 
class ClampWithGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, min, max):
        ctx.min = min
        ctx.max = max
        ctx.save_for_backward(input)
        return input.clamp(min, max)
 
    @staticmethod
    def backward(ctx, grad_in):
        input, = ctx.saved_tensors
        return grad_in * (grad_in * (input - input.clamp(ctx.min, ctx.max)) >= 0), None, None
 
 
clamp_with_grad = ClampWithGrad.apply
 
 
def vector_quantize(x, codebook):
    d = x.pow(2).sum(dim=-1, keepdim=True) + codebook.pow(2).sum(dim=1) - 2 * x @ codebook.T
    indices = d.argmin(-1)
    x_q = F.one_hot(indices, codebook.shape[0]).to(d.dtype) @ codebook
    return replace_grad(x_q, x)
 
 
class Prompt(nn.Module):
    def __init__(self, embed, weight=1., stop=float('-inf')):
        super().__init__()
        self.register_buffer('embed', embed)
        self.register_buffer('weight', torch.as_tensor(weight))
        self.register_buffer('stop', torch.as_tensor(stop))
 
    def forward(self, input):
        input_normed = F.normalize(input.unsqueeze(1), dim=2)
        embed_normed = F.normalize(self.embed.unsqueeze(0), dim=2)
        dists = input_normed.sub(embed_normed).norm(dim=2).div(2).arcsin().pow(2).mul(2)
        dists = dists * self.weight.sign()
        return self.weight.abs() * replace_grad(dists, torch.maximum(dists, self.stop)).mean()
 
 
def parse_prompt(prompt):
    vals = prompt.rsplit(':', 2)
    vals = vals + ['', '1', '-inf'][len(vals):]
    return vals[0], float(vals[1]), float(vals[2])
 
 
class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow
        self.augs = nn.Sequential(
            K.RandomHorizontalFlip(p=0.5),
            # K.RandomSolarize(0.01, 0.01, p=0.7),
            K.RandomSharpness(0.3,p=0.4),
            K.RandomAffine(degrees=30, translate=0.1, p=0.8, padding_mode='border'),
            K.RandomPerspective(0.2,p=0.4),
            K.ColorJitter(hue=0.01, saturation=0.01, p=0.7))
        self.noise_fac = 0.1


    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        for _ in range(self.cutn):
            size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
            offsetx = torch.randint(0, sideX - size + 1, ())
            offsety = torch.randint(0, sideY - size + 1, ())
            cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
            cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))
        batch = self.augs(torch.cat(cutouts, dim=0))
        if self.noise_fac:
            facs = batch.new_empty([self.cutn, 1, 1, 1]).uniform_(0, self.noise_fac)
            batch = batch + facs * torch.randn_like(batch)
        return batch
 
 
def load_vqgan_model(config_path, checkpoint_path):
    config = OmegaConf.load(config_path)
    if config.model.target == 'taming.models.vqgan.VQModel':
        model = vqgan.VQModel(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    elif config.model.target == 'taming.models.cond_transformer.Net2NetTransformer':
        parent_model = cond_transformer.Net2NetTransformer(**config.model.params)
        parent_model.eval().requires_grad_(False)
        parent_model.init_from_ckpt(checkpoint_path)
        model = parent_model.first_stage_model
    elif config.model.target == 'taming.models.vqgan.GumbelVQ':
        model = vqgan.GumbelVQ(**config.model.params)
        print(config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    else:
        raise ValueError(f'unknown model type: {config.model.target}')
    del model.loss
    return model
 
 
def resize_image(image, out_size):
    ratio = image.size[0] / image.size[1]
    area = min(image.size[0] * image.size[1], out_size[0] * out_size[1])
    size = round((area * ratio)**0.5), round((area / ratio)**0.5)
    return image.resize(size, Image.LANCZOS)

def download_img(img_url):
    try:
        return wget.download(img_url,out="input.jpg") # type: ignore
    except:
        return

print("Step 4. done")


In [ ]:
#@title <center>Action Block

from google.colab import files


print("5. Give your text prompts and settings here")


#@markdown <br> <center> <b>INSTRUCTION: Fill the form then run this Block

#@markdown <br><center><p><b>Enter your text prompt to generate images</b>
#@markdown <br> (separated with | )
#@markdown <br> You can also set weights with a <b>: (colon)</b>
#@markdown <br> <p><i>ice forest :8 | trending on Artstation :9




input_text_for_image_generation = "fan on planet | artstation" #@param {type:"string"}

#@markdown <br><br><center><p>Set Width and Height of output images

width_of_output_image =  300#@param {type:"number"}
height_of_output_image =  150#@param {type:"number"}

#@markdown <br><br><center>
#<p>Select only the models that are downloaded in step 3
model_to_use = "vqgan_imagenet_f16_16384" # @ param ["vqgan_imagenet_f16_16384", "vqgan_imagenet_f16_1024", "wikiart_1024", "wikiart_16384", "coco", "faceshq", "sflckr", "ade20k", "ffhq", "celebahq", "gumbel_8192"]

#@markdown <br><br><center><p>Select an image to enable the Algorithm to start creating. Transparent png images with few dots work best.
initial_image = "picture.png" #@param {type:"string"}
if initial_image=="picture.png" or "-1" or "0":
  !wget -O picture.png https://www.dropbox.com/s/f75ts4p736s9560/sample_png_4.png?dl=0


#@markdown <center><p><br><br>Steer your generated image towards this image

target_images = "None" #@param {type:"string"}

#@markdown <br><br><center><p>-1 for random value<br>
#@markdown Giving the same value (from 1 to 65536) every time, with same parameters will produce similar images 
seed = 20#@param {type:"number"}

#@markdown <br><br><center><p>-1 for infinite output.
#@markdown <br>any other value (recommended 400) >0 is valid
#@markdown <br>Generally, higher this value, better the images 

max_iterations =  105#@param {type:"number"}




input_images = ""


#simplified:
textos = input_text_for_image_generation

ancho=width_of_output_image
alto = height_of_output_image
modelo = model_to_use
interval_images = 1


download_step=100

nombres_modelos={"vqgan_imagenet_f16_16384": 'ImageNet 16384',"vqgan_imagenet_f16_1024":"ImageNet 1024", 
                 "wikiart_1024":"WikiArt 1024", "wikiart_16384":"WikiArt 16384", "coco":"COCO-Stuff", "faceshq":"FacesHQ", "sflckr":"S-FLCKR", "ade20k":"ADE20K", "ffhq":"FFHQ", "celebahq":"CelebA-HQ", "gumbel_8192": "Gumbel 8192"}
nombre_modelo = nombres_modelos[modelo]     

if modelo == "gumbel_8192":
    is_gumbel = True
else:
    is_gumbel = False

if seed == -1:
    seed = None
if initial_image == "None":
    initial_image = None
elif initial_image and initial_image.lower().startswith("http"):
    initial_image = download_img(initial_image)


if target_images == "None" or not target_images:
    target_images = []
else:
    target_images = target_images.split("|")
    target_images = [image.strip() for image in target_images]

if initial_image or target_images != []:
    input_images = True

textos = [frase.strip() for frase in textos.split("|")]
if textos == ['']:
    textos = []


args = argparse.Namespace(
    prompts=textos,
    image_prompts=target_images,
    noise_prompt_seeds=[],
    noise_prompt_weights=[],
    size=[ancho, alto],
    init_image=initial_image,
    init_weight=0.,
    clip_model='ViT-B/32',
    vqgan_config=f'{modelo}.yaml',
    vqgan_checkpoint=f'{modelo}.ckpt',
    step_size=0.1,
    cutn=64,
    cut_pow=1.,
    display_freq=interval_images,
    seed=seed,
)

print("5. done")

print("6. [Final step] Do the execution...")
#@markdown <br>

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

last_i=-1

if textos:
    print('Using texts:', textos)
if target_images:
    print('Using image prompts:', target_images)
if args.seed is None:
    seed = torch.seed()
else:
    seed = args.seed
torch.manual_seed(seed)
print('Using seed:', seed)

model = load_vqgan_model(args.vqgan_config, args.vqgan_checkpoint).to(device)
perceptor = clip.load(args.clip_model, jit=False)[0].eval().requires_grad_(False).to(device)

cut_size = perceptor.visual.input_resolution
if is_gumbel:
    e_dim = model.quantize.embedding_dim
else:
    e_dim = model.quantize.e_dim

f = 2**(model.decoder.num_resolutions - 1)
make_cutouts = MakeCutouts(cut_size, args.cutn, cut_pow=args.cut_pow)
if is_gumbel:
    n_toks = model.quantize.n_embed
else:
    n_toks = model.quantize.n_e

toksX, toksY = args.size[0] // f, args.size[1] // f
sideX, sideY = toksX * f, toksY * f
if is_gumbel:
    z_min = model.quantize.embed.weight.min(dim=0).values[None, :, None, None]
    z_max = model.quantize.embed.weight.max(dim=0).values[None, :, None, None]
else:
    z_min = model.quantize.embedding.weight.min(dim=0).values[None, :, None, None]
    z_max = model.quantize.embedding.weight.max(dim=0).values[None, :, None, None]

if args.init_image:
    pil_image = Image.open(args.init_image).convert('RGB')
    pil_image = pil_image.resize((sideX, sideY), Image.LANCZOS)
    z, *_ = model.encode(TF.to_tensor(pil_image).to(device).unsqueeze(0) * 2 - 1)
else:
    one_hot = F.one_hot(torch.randint(n_toks, [toksY * toksX], device=device), n_toks).float()
    if is_gumbel:
        z = one_hot @ model.quantize.embed.weight
    else:
        z = one_hot @ model.quantize.embedding.weight
    z = z.view([-1, toksY, toksX, e_dim]).permute(0, 3, 1, 2)
z_orig = z.clone()
z.requires_grad_(True)
opt = optim.Adam([z], lr=args.step_size)

normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                 std=[0.26862954, 0.26130258, 0.27577711])

pMs = []

for prompt in args.prompts:
    txt, weight, stop = parse_prompt(prompt)
    embed = perceptor.encode_text(clip.tokenize(txt).to(device)).float()
    pMs.append(Prompt(embed, weight, stop).to(device))

for prompt in args.image_prompts:
    path, weight, stop = parse_prompt(prompt)
    img = resize_image(Image.open(path).convert('RGB'), (sideX, sideY))
    batch = make_cutouts(TF.to_tensor(img).unsqueeze(0).to(device))
    embed = perceptor.encode_image(normalize(batch)).float()
    pMs.append(Prompt(embed, weight, stop).to(device))

for seed, weight in zip(args.noise_prompt_seeds, args.noise_prompt_weights):
    gen = torch.Generator().manual_seed(seed)
    embed = torch.empty([1, perceptor.visual.output_dim]).normal_(generator=gen)
    pMs.append(Prompt(embed, weight).to(device))

def synth(z):
    if is_gumbel:
        z_q = vector_quantize(z.movedim(1, 3), model.quantize.embed.weight).movedim(3, 1)
    else:
        z_q = vector_quantize(z.movedim(1, 3), model.quantize.embedding.weight).movedim(3, 1)
    
    return clamp_with_grad(model.decode(z_q).add(1).div(2), 0, 1)

def add_xmp_data(nombrefichero):
    imagen = ImgTag(filename=nombrefichero)
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'creator', 'VQGAN+CLIP', {"prop_array_is_ordered":True, "prop_value_is_array":True})
    if args.prompts:
        imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', " | ".join(args.prompts), {"prop_array_is_ordered":True, "prop_value_is_array":True})
    else:
        imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', 'None', {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'i', str(i), {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'model', nombre_modelo, {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'seed',str(seed) , {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'input_images',str(input_images) , {"prop_array_is_ordered":True, "prop_value_is_array":True})
    #for frases in args.prompts:
    #    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'Prompt' ,frases, {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.close()

def add_stegano_data(filename):
    data = {
        "title": " | ".join(args.prompts) if args.prompts else None,
        "notebook": "VQGAN+CLIP",
        "i": i,
        "model": nombre_modelo,
        "seed": str(seed),
        "input_images": input_images
    }
    lsb.hide(filename, json.dumps(data)).save(filename)

@torch.no_grad()
def checkin(i, losses):
    global last_i,download_step
    last_i=i

    

    if i==download_step:
      download_step=download_step+100
      try:
        filename = f"steps/{last_i:04}.png"
        files.download(filename)
        print("downloading"+filename)
      except Exception as e:
        print("Exc: "+str(e))

    losses_str = ', '.join(f'{loss.item():g}' for loss in losses)
    tqdm.write(f'\n\n\nPrompt: {textos}\nIteration number: {i}/{max_iterations}')
    #\nloss: {sum(losses).item():g}, losses: {losses_str}')
    out = synth(z)
    TF.to_pil_image(out[0].cpu()).save('progress.png')
    add_stegano_data('progress.png')
    add_xmp_data('progress.png')
    display.display(display.Image('progress.png'))

def ascend_txt():
    global i
    out = synth(z)
    iii = perceptor.encode_image(normalize(make_cutouts(out))).float()

    result = []

    if args.init_weight:
        result.append(F.mse_loss(z, z_orig) * args.init_weight / 2)

    for prompt in pMs:
        result.append(prompt(iii))
    img = np.array(out.mul(255).clamp(0, 255)[0].cpu().detach().numpy().astype(np.uint8))[:,:,:]
    img = np.transpose(img, (1, 2, 0))
    filename = f"steps/{i:04}.png"
    imageio.imwrite(filename, np.array(img))
    add_stegano_data(filename)
    add_xmp_data(filename)
    return result

def train(i):
    opt.zero_grad()
    lossAll = ascend_txt()
    if i % args.display_freq == 0:
        checkin(i, lossAll)
    loss = sum(lossAll)
    loss.backward()
    opt.step()
    with torch.no_grad():
        z.copy_(z.maximum(z_min).minimum(z_max))

i = 0
try:
    with tqdm() as pbar:
        while True:
            train(i)
            if i == max_iterations:
                break
            i += 1
            pbar.update()
except KeyboardInterrupt:
    pass
print("end of execution block")


import re

#creating name of file from prompts
op_file_name = re.sub(r"\|","_",input_text_for_image_generation)
op_file_name = re.sub(r"\s", "_", op_file_name)
op_file_name = re.sub(r"\__", "_", op_file_name)
op_file_name = re.sub(r":", "", op_file_name)

#@markdown <br> <center> <b> Picture output settings

#@markdown <p>Typically you'd skip the first few frames and start from say frame 10 or 20 if you've set a total iteration of 400
#@markdown <p>To start from the beginning keep it at 0
#This is the frame where the video will start
no_of_frames_to_skip= 0 #@param {type:"number"}
starting_frame = no_of_frames_to_skip

#@markdown <p>
no_of_images_to_download = 10#@param {type:"number"}
#no_of_images_to_download=no_of_images_to_download-1

last_frame=last_i
step = int(last_frame/no_of_images_to_download)
print(f"starting_frame {starting_frame}")
print(f"last_frame : {last_frame}")
print(f"step : {step}")

filenames_to_download=""
for i in range ((starting_frame+step),last_frame+1,step):
    filename = f"steps/{i:04}.png"
    #files.download(filename)
    filenames_to_download+=filename+" "

import os
try:
  print(filenames_to_download)
  zip_filename=op_file_name+".zip"
  if os.path.exists(zip_filename):
    os.remove(zip_filename)
  os.system(f"zip {zip_filename} {filenames_to_download}")
  #files.download(zip_filename)
except Exception as e:
  print("Error while zipping")


#@markdown <p> <br><br><br>
#@markdown <br> <center> <b> Video output settings




from tqdm.notebook import tqdm
init_frame = 1 #This is the frame where the video will start

last_frame = last_i #You can change i to the number of the last frame you want to generate. 
#It will raise an error if that number of frames does not exist.

min_fps = 1#@param {type:"number"}


max_fps = 30#@param {type:"number"}

total_frames = last_frame-init_frame


length_of_video_in_secs = 15 #@param {type:"number"}
#Desired time of the video in seconds
length=length_of_video_in_secs

frames = []
tqdm.write('Generating video....')
for i in range(0,last_i): #
    filename = f"steps/{i:04}.png"
    frames.append(Image.open(filename))

#fps = last_frame/10
fps = np.clip(total_frames/length,min_fps,max_fps)



from subprocess import Popen, PIPE

op_vid_name=op_file_name+".mp4"

p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', op_vid_name], stdin=PIPE)
for im in tqdm(frames):
    im.save(p.stdin, 'PNG')
p.stdin.close()

print("The video is now being compressed, wait...")
p.wait()
print("video is ready to download")

from google.colab import files
#files.download(op_vid_name)
print("Downloading")
print("ALL DONE")




___________

_______________________________

<h1><b><center>Process 2/2: Upscaler

<h1>Init

In [ ]:
!pwd
%cd home_of_project

In [ ]:
!nvidia-smi
!pwd


!pip3 install virtualenv
!virtualenv newtestenv
!source /content/newtestenv/bin/activate; pip3 list


%bookmark home_of_project /content


# Clone Real-ESRGAN and enter the Real-ESRGAN
!source /content/newtestenv/bin/activate; git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN

# Set up the environment
!source /content/newtestenv/bin/activate;pip install basicsr
!source /content/newtestenv/bin/activate;pip install facexlib
!source /content/newtestenv/bin/activate;pip install gfpgan
!source /content/newtestenv/bin/activate;pip install -r requirements.txt
!source /content/newtestenv/bin/activate;python setup.py develop
# Download the pre-trained model
!source /content/newtestenv/bin/activate; wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models


<center> <h1><b>Upload

In [ ]:
#needed!!
%bookmark home_of_project /content
!pwd
%bookmark -l
%cd home_of_project


In [ ]:

%cd home_of_project 
!pwd

import shutil
upload_folder = 'Real-ESRGAN/upload'
result_folder = 'Real-ESRGAN/results'
#copy the last picture
src_fname=f"{last_i:04}.png"
src_fname_and_path=f"steps/{last_i:04}.png"
dst_path=os.path.join(upload_folder, src_fname)
print(src_fname_and_path)
print(dst_path)

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)

os.mkdir(upload_folder)
os.mkdir(result_folder)

print(f'copy {src_fname_and_path} to {upload_folder}')
shutil.copy(src_fname_and_path, upload_folder)

<center> <h1><b>Run Now

In [ ]:
#@title Set enlarging factor

#needed!!
print(op_file_name)
!pwd
%cd Real-ESRGAN/
scale = 3 #@param {type:"slider", min:0, max:10, step:0.5}


In [ ]:
!source /content/newtestenv/bin/activate; python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --outscale scale --face_enhance


In [ ]:
!pwd

In [ ]:
#@title Download a zipped copy of the last frame 
# # @markdown Run this cell to connect to your gdrive and copy your results to the foldername you set here
zip_filename_enhanced = op_file_name+"_enhanced.zip"
if os.path.exists(zip_filename_enhanced):
  os.remove(zip_filename_enhanced)
os.system(f"zip -r -j {zip_filename_enhanced} results/*")
files.download(zip_filename_enhanced)

In [ ]:
#@title If you give permission, <br> you can also copy the video and zip results of the selected photos and the Enhanced image. 
#@markdown Run this cell to connect to your gdrive and copy your results to the foldername you set here

drive_folder_name = '0ColabOutputs' #@param {type:"string"}
os.mkdir(f"gdrive/MyDrive/{drive_folder_name}/")
!pwd
from google.colab import drive

drive.mount('/content/Real-ESRGAN/gdrive')

zip_filename_enhanced = op_file_name+"_enhanced.zip"
shutil.copy(zip_filename_enhanced, f"gdrive/MyDrive/{drive_folder_name}/") # image enhancer

%cd home_of_project
shutil.copy(op_vid_name, f"Real-ESRGAN/gdrive/MyDrive/{drive_folder_name}/")
shutil.copy(zip_filename,f"Real-ESRGAN/gdrive/MyDrive/{drive_folder_name}/")
